In [1]:
import tlsh
import random
import numpy as np

In [188]:
def get_rnd_string_and_hash():
    my_string = "1234"
    slots = 50
    # a_pad = 'a'*1000
    new_string = ''
    binary_representation = bytes()

    for j in range(slots):
        new_string = new_string + random.choice(my_string)

    binary_representation = new_string.encode()
    secret_hashed = tlsh.oldhash(binary_representation)
#     print(new_string)

    #####################################################
    # TRY
    # new_string = a_pad + new_string + a_pad
    # binary_representation = new_string.encode()

    # secret_hashed_2 = tlsh.oldhash(binary_representation)
    # print(new_string)
    return (new_string, secret_hashed)
    #####################################################

    # print(tlsh.diff(secret_hashed, secret_hashed_2))

In [189]:
import csv
def get_matches_below_thr(threshold: int, secret_hashed: str, strings_file: str):
    
    diff_n = 0
    num = 0

    list_of_matches = []
    list_of_hashes = []
    list_of_diff = []
    with open(strings_file, newline='') as csvfile:
        reader_obj = csv.reader(csvfile, delimiter=',', quotechar='|')

        if(not secret_hashed == 'TNULL'):
            for row in reader_obj:
                if(not row[1] == 'TNULL'):
                    diff_n = tlsh.diff(secret_hashed, row[1])
                    if(diff_n < threshold):
                        num += 1
                        list_of_matches.append(row[0])
                        list_of_hashes.append(row[1])
                        list_of_diff.append(diff_n)

    return (num, list_of_matches, list_of_hashes, list_of_diff)

In [201]:
s_and_h = get_rnd_string_and_hash()
secret_string = s_and_h[0]
ss_hash = s_and_h[1]

In [202]:
matches = get_matches_below_thr(80, ss_hash, "string_hashing.txt")

In [203]:
diff_array = np.zeros((len(matches[1]), len(matches[1])))

for i in range(len(matches[1])):
    for j in range(i, len(matches[1])):
        b1 = matches[1][i].encode()
        b2 = matches[1][j].encode()
        diff_array[i][j] = tlsh.diff(tlsh.oldhash(b1), tlsh.oldhash(b2))

diff_vector = np.array(matches[3])
print("Diff array:\n", diff_array, "\n")
print("Diff vector:\n", diff_vector.T, "\n")
print("Shape of diff_array: ", np.shape(diff_array))
print('='*80)

eligible_parents_check = []

for i in range(len(matches[1])):
    for j in range(i, len(matches[1])):
        if(diff_array[i][j] > diff_vector[i] or diff_array[i][j] > diff_vector[j]):
            eligible_parents_check.append((secret_string, matches[1][i], matches[1][j], i, j))

Diff array:
 [[  0. 112. 101. ...  76. 103. 109.]
 [  0.   0. 103. ... 114. 105. 109.]
 [  0.   0.   0. ...  83.  94.  83.]
 ...
 [  0.   0.   0. ...   0.  81.  96.]
 [  0.   0.   0. ...   0.   0. 101.]
 [  0.   0.   0. ...   0.   0.   0.]] 

Diff vector:
 [73 75 75 79 79 75 79 79 79 77 73 77 72 75 79 74 78 76 76 76 79 79 74 75
 79 73 78 79 73 78 77 78 79 79 79 77 75 79 73 76 76 73 79 79 76 76 75 79
 78 71 79 76 75 78 76 75 79 79 78 78 76 78 79 79 73 77 77 76 78 77 78 79
 73 79 73] 

Shape of diff_array:  (75, 75)


In [204]:
def find_diff_string(st_1: str, st_2: str):
    diff_str = ''
    for i in range(len(st_1)):
        if(st_1[i] == st_2[i]):
            diff_str += ' '
        else:
            diff_str += 'D'
    return diff_str

def mutate(probability: float, start_idx: int, end_idx: int, inp_str: str, all_char_str: str):
    inp_list = list(inp_str)
    for idx in range(start_idx, end_idx+1):
        test = random.random()
        if(test <= probability):
            while(1):
                new_char = random.choice(all_char_str)
                if(new_char != inp_list[idx]):
                    inp_list[idx] = new_char
                    break
    return ''.join(inp_list)

def crossover(string1: str, string2: str, start_idx:int, end_idx: int, all_char_str: str):
    return (string1[0:start_idx] + string2[start_idx:end_idx+1] + string1[end_idx+1::],
            string2[0:start_idx] + string1[start_idx:end_idx+1] + string2[end_idx+1::])

In [205]:
str_idxs_and_sims = []

for ss, p1, p2, i, j in eligible_parents_check:
    if(not p1 == p2):
        a = find_diff_string(ss, p1)
        print(a, "- NOT VISIBLE TO PLAYER")
        b = find_diff_string(ss, p2)
        print(b, "- NOT VISIBLE TO PLAYER")
        c = find_diff_string(p1, p2)
        print(c)
        
        count_of_spaces = [0,0,0]
        for idx in range(len(a)):
            if(a[idx] == ' '):
                count_of_spaces[0] += 1
            if(b[idx] == ' '):
                count_of_spaces[1] += 1
            if(c[idx] == ' '):
                count_of_spaces[2] += 1
        str_idxs_and_sims.append(((i,j),tuple(count_of_spaces), (diff_vector[i], diff_vector[j], diff_array[i][j])))

DDD DDDD   D  DDDD DDDDD  D DDDD DD DDD D D DDDDDD - NOT VISIBLE TO PLAYER
   D  DDDDD D DDD   D  D  D  DDD DDD DDD DDDDD   D - NOT VISIBLE TO PLAYER
DDDDDD  DDDDD DDDD DDDDD  D DD D  DDDD DDDDDD DDDD
DDD DDDD   D  DDDD DDDDD  D DDDD DD DDD D D DDDDDD - NOT VISIBLE TO PLAYER
 DDDDDDDDDDDDDDDDD D  D DDDDDDDDD D D D DDD DDD DD - NOT VISIBLE TO PLAYER
DDDDDD  DDDDDD DD  DDD DDD DD DDDDD  DD DDD DDDDD 
DDD DDDD   D  DDDD DDDDD  D DDDD DD DDD D D DDDDDD - NOT VISIBLE TO PLAYER
DDDDD DD DDDDDDDD DDD D DDDD D DDD DDDDDDDD DDDDDD - NOT VISIBLE TO PLAYER
DD DDDDD DDDDDDD DDDDD DDD DDDDDD DD  DDDD   D  DD
DDD DDDD   D  DDDD DDDDD  D DDDD DD DDD D D DDDDDD - NOT VISIBLE TO PLAYER
DDD  DDDDDDDD DDDDD DDDDDDDDDDDDDDDDDDDDD D DDDD D - NOT VISIBLE TO PLAYER
DDD D  DDDDDD D DDDDDD DDDDDD D DD DDD DD D   DDDD
DDD DDDD   D  DDDD DDDDD  D DDDD DD DDD D D DDDDDD - NOT VISIBLE TO PLAYER
 DDDD  DDDDDDD DD  DD DDDDDD DD DDDDDDDDDDD DDDDDD - NOT VISIBLE TO PLAYER
DD D DD DDDDDDDDDD   DDDDD DDDDDDDDDDDDDDDD DD

In [206]:
###################
# elem -> (index p1, index p2), (simil p1 & s, simil p2 & s, simil p1 & p2), (diff ss & p1, diff ss & p2)
# Want to maximize similarity between p1 & s, p2 & s, while minimizing between p1 & p2
# Minimizing, can be done directly
# Maximizing, ??? - Minimizing difference?
###################

def key_for_pvp_sim(element):
    return element[1][2]

sorted_sias = sorted(str_idxs_and_sims, key=key_for_pvp_sim, reverse = False)

pvp_similarity_upper_bound = 10
pvs_diff_upper_bound = 100

for elem in sorted_sias:
    idx = sorted_sias.index(elem)
    if(
    elem[1][2] > pvp_similarity_upper_bound or
    (elem[2][1] > pvs_diff_upper_bound or
    elem[2][2] > pvs_diff_upper_bound)):
        sorted_sias[idx] = 0
        
while(1):    
    try:
        (sorted_sias.index(0))
        sorted_sias.pop(sorted_sias.index(0))
    except:
        break

In [207]:
def find_D_blocks(least_idx: int, diff_str: str):    
    block_list = []
    
    is_block = False
    start_idx = None
    end_idx = None
    
    for idx in range(len(diff_str)):
        if(not is_block):
            if(diff_str[idx] == 'D'):
                is_block = True
                start_idx = idx
                end_idx = idx
            else:
                continue
        else:
            if(diff_str[idx] == 'D'):
                end_idx = idx
            else:
                is_block = False
                block_list.append((start_idx, end_idx))
                start_idx = None
                end_idx = None
    
    if(is_block):
        block_list.append((start_idx, end_idx))
            
    return block_list

In [208]:
n_parent_tuples = 0
parent_threshold = 1000
children = []
for sias in sorted_sias:
#     print(sias)
    if(n_parent_tuples > parent_threshold):
        break
    
    n_parent_tuples += 1
    
    p1 = matches[1][sias[0][0]]
    p2 = matches[1][sias[0][1]]
    
    sim_p1_p2 = sias[1][2]
    diff_p1_s = sias[2][0]
    diff_p2_s = sias[2][1]
    diff_p1_p2 = sias[2][2]
    
    diff_string = find_diff_string(p1, p2)
#     print(diff_string)
    least_idx = 0
    block_list = find_D_blocks(least_idx, diff_string)
    
    for s_e in block_list:
        start = s_e[0]
        end = s_e[1]
        
        c1, c2 = crossover(p1, p2, start, end, '1234')
        c1_hash = tlsh.oldhash(c1.encode())
        c2_hash = tlsh.oldhash(c2.encode())
        
        if(c1_hash != 'TNULL'):
            diff_c1_s = tlsh.diff(ss_hash, c1_hash)
            d_string_p1 = find_diff_string(secret_string, p1)
            d_string_c1 = find_diff_string(secret_string, c1)
            if(diff_c1_s < diff_p1_s):
                children.append((c1, 
                                 diff_p1_s - diff_c1_s, 
                                 diff_p1_s, 
                                 diff_c1_s, 
                                 d_string_p1.count('D'), 
                                 d_string_c1.count('D'))
                               )
#                 print(start, end, "1", diff_c1_s, diff_p1_s)
            
        if(c2_hash != 'TNULL'):
            diff_c2_s = tlsh.diff(ss_hash, c2_hash)
            d_string_p2 = find_diff_string(secret_string, p2)
            d_string_c2 = find_diff_string(secret_string, c2)
            if(diff_c2_s < diff_p2_s):
                children.append((c2, 
                                 diff_p2_s - diff_c2_s, 
                                 diff_p2_s, 
                                 diff_c2_s, 
                                 d_string_p2.count('D'), 
                                 d_string_c2.count('D'))
                               )
#                 print(start, end, "2", diff_c2_s, diff_p2_s)
        
print(len(children))

1474


In [209]:
def children_sort_key(inp_tup):
    return inp_tup[1]

In [210]:
sorted_children = sorted(children, key=children_sort_key, reverse = True)
for i in range(1000):
    print(sorted_children[i], i)
    if(sorted_children[i][1] == 9):
        break
#     pass
    
# x1 = sorted_children[0][0].encode()
# x2 = sorted_children[1][0].encode()

# print(tlsh.diff(tlsh.oldhash(x1), tlsh.oldhash(x2)))
# print(tlsh.diff(tlsh.oldhash(x1), ss_hash))
# print(tlsh.diff(tlsh.oldhash(x2), ss_hash))

('33424233441211141213343232422142433411134114231131', 19, 78, 59, 34, 37) 0
('14233444424111132332331413212342423342233421413341', 17, 79, 62, 40, 39) 1
('13143124344134242331244111111324323314141433414124', 17, 79, 62, 38, 40) 2
('22413424142334242414333413441213141311212341111411', 16, 77, 61, 37, 37) 3
('34241311123234412132143242424342131133123144414113', 16, 76, 60, 33, 32) 4
('33214111113414331112123144424213241324233342412211', 15, 77, 62, 37, 35) 5
('42141413333442324243231444211111124332423323111113', 15, 79, 64, 37, 31) 6
('34243321344241331411413212424214414142113324243211', 14, 73, 59, 41, 41) 7
('31242121131423342412424334222432111342433423344222', 14, 76, 62, 37, 39) 8
('31431422111133242341424242412424332111132324111112', 14, 79, 65, 43, 42) 9
('23223141312324234441313112232333243241412424211111', 14, 79, 65, 38, 40) 10
('34241311134111142132143242424342131133242334414113', 14, 76, 62, 33, 36) 11
('14233412211324244212331413212342423342111134242411', 14, 79, 65, 40, 40)

In [216]:
get_matches_below_thr(58, tlsh.oldhash('12343331121213141244431242434332331224213414412211'.encode()), 'string_hashing.txt')
# get_matches_below_thr(70, ss_hash, 'string_hashing.txt')
'12343331121213141(4131241131134432)31224213414412211'
'12343331121213141(2444312424343323)31224213414412211'

(1,
 ['12343331121213141413124113113443231224213414412211'],
 ['C9900284A54464002C84E3A21304589A2322242934038492050687151A4B90F4122002'],
 [56])